In [2]:
import numpy as np
import pandas as pd


In [4]:
book=pd.read_csv("C:/Users/hp/Downloads/BX-Books.csv",sep=';',error_bad_lines=False,encoding='latin-1')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\hp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2

In [5]:
book.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [6]:
book=book[['ISBN','Book-Title','Book-Author']]

In [7]:
book.shape

(271360, 3)

In [8]:
book.head(2)

,ISBN,Book-Title,Book-Author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright


In [9]:
book.rename(columns={'Book-Title':'Book Name','Book-Author':'Author'},inplace=True)

In [10]:
rating=pd.read_csv("C:/Users/hp/Downloads/BX-Book-Ratings.csv",sep=';',encoding='latin-1')

In [11]:
rating.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [12]:
rating.rename(columns={'User-ID':'user_id','Book-Rating':'rating'},inplace=True)

In [13]:
rating.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [14]:
book.head()

,ISBN,Book Name,Author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [15]:
count=rating['user_id'].value_counts()
rating=rating[rating['user_id'].isin(count[count>200].index)]
len(rating)

526356

In [16]:
rating_with_bookname=rating.merge(book,on='ISBN')

In [17]:
rating_with_bookname.head(2)

,user_id,ISBN,rating,Book Name,Author
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner


In [18]:
book_num_rating=rating_with_bookname.groupby('Book Name')['rating'].agg('count').reset_index()
book_num_rating.head(2)

,Book Name,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1


In [19]:
salman=rating_with_bookname.merge(book_num_rating,on='Book Name')
salman.head(2)

,user_id,ISBN,rating_x,Book Name,Author,rating_y
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,82


In [20]:
srk=salman[salman['rating_y']>=50]

In [21]:
len(srk)

61853

In [22]:
aamir=srk.pivot_table(index='Book Name',columns='user_id',values='rating_x').fillna(0)

In [23]:
aamir.shape

(742, 888)

In [24]:
from scipy.sparse import csr_matrix
ranbir=csr_matrix(aamir)

In [25]:
type(aamir)

pandas.core.frame.DataFrame

In [26]:
ranbir

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 15226 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.neighbors import NearestNeighbors
nn=NearestNeighbors(metric='cosine',algorithm='brute')

In [28]:
nn.fit(ranbir)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [29]:
instance,books=nn.kneighbors(aamir.iloc[26,:].values.reshape(1,888),n_neighbors=6)

In [30]:
books=books.ravel()
books

array([ 26,  87, 348, 461, 629,  64], dtype=int64)

In [31]:
for i in books:
    print(aamir.index[i])

A Virtuous Woman (Oprah's Book Club (Paperback))
Blue Diary
Moo
Slammerkin
The Reader
Back Roads


In [32]:
np.where(aamir.index=='Animal Farm')[0][0]

54

In [33]:
def suggest(name):
    book_id=np.where(aamir.index==name)[0][0]
    instances,books=nn.kneighbors(aamir.iloc[book_id,:].reshape(1,888),n_neighbors=6)
    for i in books.ravel():
        print(aamir.index[i])

In [70]:
name=input('Enter book name:')
suggest(name)

Enter book name:We'll Meet Again
We'll Meet Again
Pretend You Don't See Her
While My Pretty One Sleeps
Daddy's Little Girl
Family Album
Five Days in Paris


C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
